In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, time
from scipy.stats import chi2
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
experiment1_1 = pd.read_csv('drive/My Drive/experiment1_1.csv')
experiment1_2 = pd.read_csv('drive/My Drive/experiment1_2.csv')
experiment2_1 = pd.read_csv('drive/My Drive/experiment2_1.csv')
experiment2_2 = pd.read_csv('drive/My Drive/experiment2_2.csv')

Посмотрим на датасеты эксперимента 1 - вроде сравнимы, по 5 тыс пользователей и не пересекаются, второй эксперимент по длительности немного больше - но, видимо, добирали нужную базу

In [4]:
experiment1_1

,client_id,user_session,timestamp,event,session
0,10505,1,2020-05-01 00:00:00.000000000,main,105051
1,10505,1,2020-05-01 00:00:00.000000000,phones,105051
2,10505,1,2020-05-01 00:00:00.000000000,phones/apple,105051
3,10505,1,2020-05-01 00:00:00.000000000,phones,105051
4,10505,1,2020-05-01 00:00:00.000000000,main,105051
...,...,...,...,...,...
352211,15504,8,2020-08-13 15:27:19.500289692,phones/apple,155048
352212,15504,8,2020-08-13 15:27:19.500289692,phones,155048
352213,15504,8,2020-08-13 15:27:19.500289692,phones/apple,155048
352214,15504,8,2020-08-13 15:27:19.500289692,phones,155048


In [5]:
experiment1_1.dtypes

client_id        int64
user_session     int64
timestamp       object
event           object
session          int64
dtype: object

In [6]:
experiment1_1['client_id'].nunique()

5000

In [7]:
experiment1_2

,client_id,user_session,timestamp,event,session
0,15505,1,2020-05-01 00:00:00.000000000,main,155051
1,15505,1,2020-05-01 00:00:00.000000000,laptops,155051
2,15505,1,2020-05-01 00:00:00.000000000,main,155051
3,15505,1,2020-05-01 00:00:00.000000000,phones,155051
4,15505,1,2020-05-01 00:00:00.000000000,lost,155051
...,...,...,...,...,...
354800,20504,5,2020-07-31 17:27:53.704342993,lost,205045
354801,20504,6,2020-09-07 12:39:43.565471364,main,205046
354802,20504,6,2020-09-07 12:39:43.565471364,laptops,205046
354803,20504,6,2020-09-07 12:39:43.565471364,main,205046


In [8]:
experiment1_2.dtypes

client_id        int64
user_session     int64
timestamp       object
event           object
session          int64
dtype: object

In [9]:
experiment1_1['client_id'].nunique()

5000

In [10]:
len(set(experiment1_1['client_id']).difference(set(experiment1_2['client_id'])))

5000

Посмотрим какие события у нас есть, чтобы понять событие, которое отвечает за конверсию. События в датасетах одинаковые (и их количества, кстати, тоже более-менее похожи), за событие конверсии будем считать payment_done

In [28]:
experiment1_1['event'].value_counts()

main                    73526
phones                  61311
phones/apple            45289
lost                    40200
laptops                 36398
item_details            25522
laptops/xiaomi/1234     18242
laptops/xiaomi          17159
phones/apple/123        15013
item_reviews             6616
support                  5260
cart                     2821
fill_user_info           1996
support/message_sent     1939
checkout                  786
payment_done              138
Name: event, dtype: int64

In [27]:
experiment1_2['event'].value_counts()

main                    74269
phones                  61593
phones/apple            45642
lost                    40378
laptops                 36829
item_details            25657
laptops/xiaomi/1234     18479
laptops/xiaomi          17324
phones/apple/123        15362
item_reviews             6485
support                  5201
cart                     2724
fill_user_info           1988
support/message_sent     1858
checkout                  862
payment_done              154
Name: event, dtype: int64

Посмотрим, какое количество уникальных пользователей сконвертировалось в группах

In [13]:
experiment1_1[experiment1_1['event']=='payment_done']['client_id'].nunique()

138

In [14]:
experiment1_2[experiment1_2['event']=='payment_done']['client_id'].nunique()

151

Посмотрим на хи квадрат (метрика, подходящая для конверсии):

In [37]:
O=np.array([138,151,4862,4849])
T=np.array([145,145,4855,4855])
D=np.sum(np.square(T-O)/T)
pvalue=chi2.sf(D, df=1)
print("distance d: {0}\np-value: {1}".format(D,pvalue))

distance d: 0.6037146205476047
p-value: 0.4371642296367684


p-value сильно больше 0,05, не можем с уверенностью сказать, что изменение страниц в эксперименте значимо повлияло на конверсию в лучшую сторону. Уточнила тут за кадром еще страницу корзины - тоже нет разницы, воронки сильно идентичные, даже невооруженным взглядом

Посмотрим на результаты второго эксперимента

In [19]:
experiment2_1

,client_id,user_session,timestamp,event,session
0,10505,1,2020-05-01 00:00:00.000000000,main,105051
1,10505,1,2020-05-01 00:00:00.000000000,phones,105051
2,10505,1,2020-05-01 00:00:00.000000000,phones/apple,105051
3,10505,1,2020-05-01 00:00:00.000000000,phones,105051
4,10505,1,2020-05-01 00:00:00.000000000,main,105051
...,...,...,...,...,...
352211,15504,8,2020-08-13 15:27:19.500289692,phones/apple,155048
352212,15504,8,2020-08-13 15:27:19.500289692,phones,155048
352213,15504,8,2020-08-13 15:27:19.500289692,phones/apple,155048
352214,15504,8,2020-08-13 15:27:19.500289692,phones,155048


In [20]:
experiment2_2

,client_id,user_session,timestamp,event,session
0,20505,1,2020-05-01 00:00:00.000000000,main,205051
1,20505,1,2020-05-01 00:00:00.000000000,laptops,205051
2,20505,1,2020-05-01 00:00:00.000000000,laptops/xiaomi,205051
3,20505,1,2020-05-01 00:00:00.000000000,laptops/xiaomi/1234,205051
4,20505,1,2020-05-01 00:00:00.000000000,item_details,205051
...,...,...,...,...,...
604668,25504,8,2020-11-15 01:14:40.752860170,laptops/xiaomi/1234,255048
604669,25504,8,2020-11-15 01:14:40.752860170,laptops/xiaomi,255048
604670,25504,8,2020-11-15 01:14:40.752860170,laptops,255048
604671,25504,8,2020-11-15 01:14:40.752860170,laptops/xiaomi,255048


In [22]:
experiment2_1['client_id'].nunique()

5000

In [23]:
experiment2_2['client_id'].nunique()

5000

In [24]:
len(set(experiment1_1['client_id']).difference(set(experiment1_2['client_id'])))

5000

In [39]:
experiment2_1['event'].value_counts()

main                    73526
phones                  61311
phones/apple            45289
lost                    40200
laptops                 36398
item_details            25522
laptops/xiaomi/1234     18242
laptops/xiaomi          17159
phones/apple/123        15013
item_reviews             6616
support                  5260
cart                     2821
fill_user_info           1996
support/message_sent     1939
checkout                  786
payment_done              138
Name: event, dtype: int64

Контрольная группа, похоже, та же

In [35]:
experiment2_1[experiment2_1['event']=='payment_done']['client_id'].nunique()

138

In [30]:
experiment2_2['event'].value_counts()

phones                  103255
main                     92897
phones/apple             86718
item_details             65636
laptops                  49822
laptops/xiaomi/1234      40998
lost                     39266
phones/apple/123         35741
laptops/xiaomi           31675
item_reviews             18064
support                  14547
cart                      8582
fill_user_info            6971
support/message_sent      6439
checkout                  3109
payment_done               953
Name: event, dtype: int64

О, что-то интересное

In [34]:
experiment2_2[experiment2_2['event']=='payment_done']['client_id'].nunique()

873

Кто-то даже пару раз покупал

In [38]:
O=np.array([138,873,4862,4127])
T=np.array([506,506,4494,4494])
D=np.sum(np.square(T-O)/T)
pvalue=chi2.sf(D, df=1)
print("distance d: {0}\np-value: {1}".format(D,pvalue))

distance d: 593.9254095491397
p-value: 3.508180411512846e-131


Мы получили какое-то бесконечно маленькое plavue. Очевидно, что тестовая страница во втором тесте отлично просто отрабатывает. Что бы мы там ни сделали, нужно вносить такие же изменения. Даже не верится, надо сроки проверить

In [45]:
experiment2_1['timestamp']=pd.to_datetime(experiment2_1['timestamp'])

In [46]:
experiment2_1['date_only']=experiment2_1['timestamp'].dt.date

In [48]:
pd.crosstab(experiment2_1.date_only, experiment2_1.client_id.nunique())

col_0,5000
date_only,
2020-05-01,8598
2020-05-02,9308
2020-05-03,9907
2020-05-04,10285
2020-05-05,10127
...,...
2020-12-26,1149
2020-12-27,1267
2020-12-28,1375


In [49]:
experiment2_2['timestamp']=pd.to_datetime(experiment2_2['timestamp'])

In [50]:
experiment2_2['date_only']=experiment2_2['timestamp'].dt.date

In [51]:
pd.crosstab(experiment2_2.date_only, experiment2_2.client_id.nunique())

col_0,5000
date_only,
2020-05-01,15946
2020-05-02,16086
2020-05-03,16869
2020-05-04,16849
2020-05-05,18391
...,...
2020-12-26,2190
2020-12-27,1881
2020-12-28,2197


Как-то в эксперименте 2_2 в 2 раза больше трафика в предновогодние дни. Если мы уверены, что ничего для этого не делали, люди просто возвращались наконец дооформить покупку, то эксперимент удачный. Но надо подумать